In [1]:
'''
Author: ChenHJ
Date: 2022-12-02 12:16:06
LastEditors: ChenHJ
LastEditTime: 2022-12-02 12:16:53
FilePath: /chenhj/0302code/CMIP_divide_SASMR_cal_detrend.ipynb
Aim: 
12/2/2022：
  1）计算CMIP6模式中历史情境下分开北部、南部路径的模拟结果
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import dask
from dask.distributed import Client

## 读取数据

### 观测

In [2]:
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)
preGPCP_JJA_detrend = ca.detrend_dim(preGPCP_JJA.copy(), "time", deg=1, demean=False)

# preAIR = xr.open_dataarray("/home/ys17-23/Extension/All_India_Rainfall_index/AIR_mmperday.nc")
# preAIR_JJA = ca.p_time(preAIR, 6, 8, True)
# preAIR_JJA = preAIR_JJA.sel(time=(preAIR_JJA.time.dt.year>=1979) & (preAIR_JJA.time.dt.year <=2014))
# preAIR_JJA_detrend = ca.detrend_dim(preAIR_JJA.copy(), "time", deg=1, demean=False)

# fpreCRU = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/cru_ts4.01_r144x72_195001-201412.nc")
# preCRU = fpreCRU["pre"].sel(time=(fpreCRU["time"].dt.year>=1979) & (fpreCRU.time.dt.year<=2014))
# preCRU_JJA = ca.p_time(preCRU, 6, 8, True)/30.67
# preCRU_JJA.attrs["units"] = "mm/day"
# preCRU_JJA_detrend = ca.detrend_dim(preCRU_JJA.copy(), "time", deg=1, demean=False)

fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
# hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
hgtERA5_ver_JJA_detrend = ca.detrend_dim(hgtERA5_ver_JJA.copy(), "time", deg=1, demean=False)
uERA5_ver_JJA = ca.p_time(uERA5, 6, 8, True)
uERA5_ver_JJA_detrend = ca.detrend_dim(uERA5_ver_JJA.copy(), "time", deg=1, demean=False)
vERA5_ver_JJA = ca.p_time(vERA5, 6, 8, True)
vERA5_ver_JJA_detrend = ca.detrend_dim(vERA5_ver_JJA.copy(), "time", deg=1, demean=False)
qERA5_ver_JJA = ca.p_time(qERA5, 6, 9, True)
qERA5_ver_JJA_detrend = ca.detrend_dim(qERA5_ver_JJA.copy(), "time", deg=1, demean=False)
spERA5_JJA = ca.p_time(spERA5, 6, 8, True)
spERA5_JJA_detrend = ca.detrend_dim(spERA5_JJA.copy(), "time", deg=1, demean=False)
wERA5_MAM, wERA5_JJA, wERA5_SON, wERA5_DJF = ca.mon_to_season(wERA5)
wERA5_MAM_detrend = ca.detrend_dim(wERA5_MAM.copy(), "time", deg=1, demean=False)
wERA5_JJA_detrend = ca.detrend_dim(wERA5_JJA.copy(), "time", deg=1, demean=False)
wERA5_SON_detrend = ca.detrend_dim(wERA5_SON.copy(), "time", deg=1, demean=False)
wERA5_DJF_detrend = ca.detrend_dim(wERA5_DJF.copy(), "time", deg=1, demean=False)
sstHad_MAM, sstHad_JJA, sstHad_SON, sstHad_DJF = ca.mon_to_season(sstHad)
sstHad_MAM_detrend = ca.detrend_dim(sstHad_MAM.copy(), "time", deg=1, demean=False)
sstHad_JJA_detrend = ca.detrend_dim(sstHad_JJA.copy(), "time", deg=1, demean=False)
sstHad_SON_detrend = ca.detrend_dim(sstHad_SON.copy(), "time", deg=1, demean=False)
sstHad_DJF_detrend = ca.detrend_dim(sstHad_DJF.copy(), "time", deg=1, demean=False)
ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)
tERA5_ver_JJA_detrend = ca.detrend_dim(tERA5_ver_JJA.copy(), "time", deg=1, demean=False)
divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))
divERA5_ver_JJA_detrend = ca.detrend_dim(divERA5_ver_JJA.copy(), "time", deg=1, demean=False)

#   calculate the waver vapor vertical intergration
ptop = 1
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= ptop) * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, ptop*100)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
integration_top = 200
uq_sl_ERA5_JJA = uERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "g m-1 s-1"
vqERA5_ver_JJA.attrs["units"] = "g m-1 s-1"
uqERA5_ver_JJA_detrend = ca.detrend_dim(uqERA5_ver_JJA.copy(), "time", deg=1, demean=False)
vqERA5_ver_JJA_detrend = ca.detrend_dim(vqERA5_ver_JJA.copy(), "time", deg=1, demean=False)
# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)
uqvqdivERA5_ver_JJA_detrend = ca.detrend_dim(uqvqdivERA5_ver_JJA.copy(), "time", deg=1, demean=False)

### historical

In [3]:
fprehis = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/pr_historical_r144x72_197901-201412.nc")
prehis_JJA = fprehis["pr"].sel(time=fprehis["time"].dt.year>=1979)
prehis_JJA.attrs["units"] = "mm/day"
prehis_JJA.attrs["standard_name"] = "precipitation"

fhgthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/zg_historical_r144x72_197901-201412.nc")
hgthis_ver_JJA = fhgthis_ver_JJA["zg"].sel(time=fhgthis_ver_JJA["time"].dt.year>=1979)
# hgthis_ver_JJA = hgthis_ver_JJA - hgthis_ver_JJA.mean(dim="lon", skipna=True)

fuhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ua_historical_r144x72_197901-201412.nc")
uhis_ver_JJA = fuhis_ver_JJA["ua"].sel(time=fuhis_ver_JJA["time"].dt.year>=1979)

fvhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/va_historical_r144x72_197901-201412.nc")
vhis_ver_JJA = fvhis_ver_JJA["va"].sel(time=fvhis_ver_JJA["time"].dt.year>=1979)

# fwhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/wap_historical_r144x72_197901-201412.nc") 
# whis_ver_JJA = fwhis_ver_JJA["wap"].sel(time=fwhis_ver_JJA["time"].dt.year>=1979)

fssthis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_MAM = fssthis_MAM["__xarray_dataarray_variable__"].sel(time=fssthis_MAM["time"].dt.year>=1979)

fssthis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_JJA = fssthis_JJA["__xarray_dataarray_variable__"].sel(time=fssthis_JJA["time"].dt.year>=1979)

fssthis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_SON = fssthis_SON["__xarray_dataarray_variable__"].sel(time=fssthis_SON["time"].dt.year>=1979)

fssthis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_DJF = fssthis_DJF["__xarray_dataarray_variable__"].sel(time=fssthis_DJF["time"].dt.year>=1979)

fthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ta_historical_r144x72_197901-201412.nc")
this_ver_JJA = fthis_ver_JJA["ta"].sel(time=fthis_ver_JJA["time"].dt.year>=1979)

fuqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_uq_dpg.nc")
uqhis_ver_JJA = fuqhis_ver_JJA["uq_dpg"].sel(time=fuqhis_ver_JJA["time"].dt.year>=1979)

fvqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_vq_dpg.nc")
vqhis_ver_JJA = fvqhis_ver_JJA["vq_dpg"].sel(time=fvqhis_ver_JJA["time"].dt.year>=1979)

fuqvqdivhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_div_uqvq_dpg.nc")
uqvqdivhis_ver_JJA = fuqvqdivhis_ver_JJA["div_uqvq"].sel(time=fuqvqdivhis_ver_JJA["time"].dt.year>=1979)

fsphis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ps_historical_r144x72_197901-201412.nc")
sphis_ver_JJA = fsphis_ver_JJA["ps"].sel(time=fsphis_ver_JJA["time"].dt.year>=1979)

divhis_ver_JJA = ca.cal_divergence(uhis_ver_JJA.sel(level=[200.0, 850.0]), vhis_ver_JJA.sel(level=[200.0, 850.0]))

### SSP585

In [4]:
fpressp585_p3 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pr_ssp585_p3_r144x72_206401-209912.nc")
pressp585_p3_JJA = fpressp585_p3["pr"].sel(time=fpressp585_p3.time.dt.year>=2064)
pressp585_p3_JJA.attrs["units"] = "mm/day"
pressp585_p3_JJA.attrs["standard_name"] = "precipitation"

fhgtssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/zg_ssp585_p3_r144x72_206401-209912.nc")
hgtssp585_p3_ver_JJA = fhgtssp585_p3_ver_JJA["zg"].sel(time=fhgtssp585_p3_ver_JJA.time.dt.year>=2064)
# hgtssp585_p3_ver_JJA = hgtssp585_p3_ver_JJA - hgtssp585_p3_ver_JJA.mean(dim="lon", skipna=True)


fussp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ua_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_ver_JJA = fussp585_p3_ver_JJA["ua"].sel(time=fussp585_p3_ver_JJA.time.dt.year>=2064)

fvssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/va_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_ver_JJA = fvssp585_p3_ver_JJA["va"].sel(time=fvssp585_p3_ver_JJA.time.dt.year>=2064)

# fwssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
# wssp585_p3_ver_JJA = fwssp585_p3_ver_JJA["wap"].sel(time=fwssp585_p3_ver_JJA.time.dt.year>=2064)

fsstssp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_MAM = fsstssp585_p3_MAM["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_MAM.time.dt.year>=2064)

fsstssp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_JJA = fsstssp585_p3_JJA["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_JJA.time.dt.year>=2064)

fsstssp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_SON = fsstssp585_p3_SON["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_SON.time.dt.year>=2064)

fsstssp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_DJF = fsstssp585_p3_DJF["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_DJF.time.dt.year>=2064)

ftssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ta_ssp585_p3_r144x72_206401-209912.nc")
tssp585_p3_ver_JJA = ftssp585_p3_ver_JJA["ta"].sel(time=ftssp585_p3_ver_JJA.time.dt.year>=2064)

divssp585_p3_ver_JJA = ca.cal_divergence(ussp585_p3_ver_JJA.sel(level=[200.0, 850.0]), vssp585_p3_ver_JJA.sel(level=[200.0, 850.0]))

fuqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_uq_dpg.nc")
uqssp585_p3_ver_JJA = fuqssp585_p3_ver_JJA["uq_dpg"].sel(time=fuqssp585_p3_ver_JJA["time"].dt.year>=2064)

fvqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_vq_dpg.nc")
vqssp585_p3_ver_JJA = fvqssp585_p3_ver_JJA["vq_dpg"].sel(time=fvqssp585_p3_ver_JJA["time"].dt.year>=2064)

fuqvqdivssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_p3_div_uqvq_dpg.nc")
uqvqdivssp585_p3_ver_JJA = fuqvqdivssp585_p3_ver_JJA["div_uqvq"].sel(time=fuqvqdivssp585_p3_ver_JJA["time"].dt.year>=2064)

fspssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ps_ssp585_p3_r144x72_206401-209912.nc")
spssp585_p3_ver_JJA = fspssp585_p3_ver_JJA["ps"].sel(time=fspssp585_p3_ver_JJA["time"].dt.year>=2064)

## 计算部分

### 计算Nino3.4 index并去趋势

In [5]:
# 计算Nino3.4 index并去趋势
ssthis_nondetrend = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/non_detrend/tos_historical_r144x72_195001-201412.nc")
sstssp585_nondetrend = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/non_detrend/tos_ssp585_r144x72_201501-209912.nc")

Nino34CMIP6_MAM, Nino34CMIP6_JJA, Nino34CMIP6_SON, Nino34CMIP6_DJF = ca.mon_to_season(ca.cal_area_weighted_mean(ca.filplonlat(xr.concat([ssthis_nondetrend.sel(time=ssthis_nondetrend.time.dt.year>=1979), sstssp585_nondetrend], "time")).loc[:,:,-5:5,-170:-120]))

Nino34CMIP6_MAM_detrend = ca.detrend_dim(Nino34CMIP6_MAM, "time", deg=1, demean=True)
Nino34CMIP6_JJA_detrend = ca.detrend_dim(Nino34CMIP6_JJA, "time", deg=1, demean=True)
Nino34CMIP6_SON_detrend = ca.detrend_dim(Nino34CMIP6_SON, "time", deg=1, demean=True)
Nino34CMIP6_DJF_detrend = ca.detrend_dim(Nino34CMIP6_DJF, "time", deg=1, demean=True)
Nino34his_MAM = ca.detrend_dim(Nino34CMIP6_MAM_detrend.sel(time=Nino34CMIP6_MAM_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_JJA = ca.detrend_dim(Nino34CMIP6_JJA_detrend.sel(time=Nino34CMIP6_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_SON = ca.detrend_dim(Nino34CMIP6_SON_detrend.sel(time=Nino34CMIP6_SON_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_DJF = ca.detrend_dim(Nino34CMIP6_DJF_detrend.sel(time=Nino34CMIP6_DJF_detrend.time.dt.year<=2014), "time", deg=1, demean=True)

Nino34ssp585_p3_MAM = ca.detrend_dim(Nino34CMIP6_MAM_detrend.sel(time=Nino34CMIP6_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_JJA = ca.detrend_dim(Nino34CMIP6_JJA_detrend.sel(time=Nino34CMIP6_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_SON = ca.detrend_dim(Nino34CMIP6_SON_detrend.sel(time=Nino34CMIP6_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_DJF = ca.detrend_dim(Nino34CMIP6_DJF_detrend.sel(time=Nino34CMIP6_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

Nino34his_MAM_nor = ca.standardize(Nino34his_MAM.copy())
Nino34his_JJA_nor = ca.standardize(Nino34his_JJA.copy())
Nino34his_SON_nor = ca.standardize(Nino34his_SON.copy())
Nino34his_DJF_nor = ca.standardize(Nino34his_DJF.copy())

Nino34ssp585_p3_MAM_nor = ca.standardize(Nino34ssp585_p3_MAM.copy())
Nino34ssp585_p3_JJA_nor = ca.standardize(Nino34ssp585_p3_JJA.copy())
Nino34ssp585_p3_SON_nor = ca.standardize(Nino34ssp585_p3_SON.copy())
Nino34ssp585_p3_DJF_nor = ca.standardize(Nino34ssp585_p3_DJF.copy())

{'standard_name': 'sea_surface_temperature', 'long_name': 'Sea Surface Temperature', 'units': 'degC', 'comment': 'Temperature of upper boundary of the liquid ocean, including temperatures below sea-ice and floating ice shelves.', 'cell_methods': 'area: mean where sea time: mean', 'cell_measures': 'area: areacello', 'history': '2019-11-08T18:45:39Z altered by CMOR: replaced missing value flag (-1e+20) with standard missing value (1e+20).'}


Before flip, lon range is [0.0, 357.5].


After flip, lon range is [-180.0, 177.5].
{'standard_name': 'sea_surface_temperature', 'long_name': 'Sea Surface Temperature', 'units': 'degC', 'comment': 'Temperature of upper boundary of the liquid ocean, including temperatures below sea-ice and floating ice shelves.', 'cell_methods': 'area: mean where sea time: mean', 'cell_measures': 'area: areacello', 'history': '2019-11-08T18:45:39Z altered by CMOR: replaced missing value flag (-1e+20) with standard missing value (1e+20).'}


### 印度半岛降水分区

In [6]:
lat = prehis_JJA.coords["lat"]
lon = prehis_JJA.coords["lon"]
# 北部
nIndia_N = 35.0
nIndia_S = 20.0
nIndia_W = 70.0
nIndia_E = 85.0
lat_nIndia_range = lat[(lat >= nIndia_S) & (lat <= nIndia_N)]
lon_nIndia_range = lon[(lon >= nIndia_W) & (lon <= nIndia_E)]

preGPCP_nIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA_detrend.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
prehis_nIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))
pressp585_p3_nIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_nIndia_range, lon=lon_nIndia_range))

# detrend
preGPCP_nIndia_JJA_detrend = ca.detrend_dim(preGPCP_nIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_nIndia_JJA_detrend = ca.detrend_dim(prehis_nIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_nIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_nIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_nIndia_JJA_nor = ca.standardize(preGPCP_nIndia_JJA_detrend)
prehis_nIndia_JJA_nor = ca.standardize(prehis_nIndia_JJA_detrend)
pressp585_p3_nIndia_JJA_nor = ca.standardize(pressp585_p3_nIndia_JJA_detrend)

# 南部
sIndia_N = 20.0
sIndia_S = 5.0
sIndia_W = 70.0
sIndia_E = 85.0
lat_sIndia_range = lat[(lat >= sIndia_S) & (lat <= sIndia_N)]
lon_sIndia_range = lon[(lon >= sIndia_W) & (lon <= sIndia_E)]

preGPCP_sIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA_detrend.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
prehis_sIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
pressp585_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))

# detrend
preGPCP_sIndia_JJA_detrend = ca.detrend_dim(preGPCP_sIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_sIndia_JJA_detrend = ca.detrend_dim(prehis_sIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_sIndia_JJA_nor = ca.standardize(preGPCP_sIndia_JJA_detrend)
prehis_sIndia_JJA_nor = ca.standardize(prehis_sIndia_JJA_detrend)
pressp585_p3_sIndia_JJA_nor = ca.standardize(pressp585_p3_sIndia_JJA_detrend)

# 全印度
India_N = 35.0
India_S = 5.0
India_W = 70.0
India_E = 85.0
lat_India_range = lat[(lat >= India_S) & (lat <= India_N)]
lon_India_range = lon[(lon >= India_W) & (lon <= India_E)]

preGPCP_India_JJA = ca.cal_area_weighted_mean(preGPCP_JJA_detrend.sel(lat=lat_India_range, lon=lon_India_range))
prehis_India_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range))
pressp585_p3_India_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range))

# detrend
preGPCP_India_JJA_detrend = ca.detrend_dim(preGPCP_India_JJA.copy(), "time", deg=1, demean=True)
prehis_India_JJA_detrend = ca.detrend_dim(prehis_India_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_India_JJA_detrend = ca.detrend_dim(pressp585_p3_India_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_India_JJA_nor = ca.standardize(preGPCP_India_JJA_detrend)
prehis_India_JJA_nor = ca.standardize(prehis_India_JJA_detrend)
pressp585_p3_India_JJA_nor = ca.standardize(pressp585_p3_India_JJA_detrend)

### 降水场回归

In [ ]:
# 降水场
(
    nIndR_his_pre_slope,
    nIndR_his_pre_intercept,
    nIndR_his_pre_rvalue,
    nIndR_his_pre_pvalue,
    nIndR_his_pre_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, prehis_JJA)

(
    sIndR_his_pre_slope,
    sIndR_his_pre_intercept,
    sIndR_his_pre_rvalue,
    sIndR_his_pre_pvalue,
    sIndR_his_pre_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, prehis_JJA)

(
    IndR_his_pre_slope,
    IndR_his_pre_intercept,
    IndR_his_pre_rvalue,
    IndR_his_pre_pvalue,
    IndR_his_pre_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, prehis_JJA)

(
    nIndR_ssp585_p3_pre_slope,
    nIndR_ssp585_p3_pre_intercept,
    nIndR_ssp585_p3_pre_rvalue,
    nIndR_ssp585_p3_pre_pvalue,
    nIndR_ssp585_p3_pre_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, pressp585_p3_JJA)

(
    sIndR_ssp585_p3_pre_slope,
    sIndR_ssp585_p3_pre_intercept,
    sIndR_ssp585_p3_pre_rvalue,
    sIndR_ssp585_p3_pre_pvalue,
    sIndR_ssp585_p3_pre_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, pressp585_p3_JJA)

(
    IndR_ssp585_p3_pre_slope,
    IndR_ssp585_p3_pre_intercept,
    IndR_ssp585_p3_pre_rvalue,
    IndR_ssp585_p3_pre_pvalue,
    IndR_ssp585_p3_pre_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, pressp585_p3_JJA)

In [ ]:
models = hgthis_ver_JJA.coords["models"]
nIndR_his_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_pre_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of his regress onto 1979-2014 "),
)
sIndR_his_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_pre_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of his regress onto 1979-2014 "),
)
IndR_his_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_pre_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of his regress onto 1979-2014 "),
)

nIndR_ssp585_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp585_p3 regress onto 1979-2014 "),
)
sIndR_ssp585_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp585_p3 regress onto 1979-2014 "),
)
IndR_ssp585_p3_pre_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_pre_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_pre_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_pre_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_pre_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_pre_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="pre fields of ssp585_p3 regress onto 1979-2014 "),
)

# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_pre_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_pre_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_pre_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_pre_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_pre_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_pre_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_pre_regress.nc")
  
nIndR_his_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_pre_regress.nc")
sIndR_his_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_pre_regress.nc")
IndR_his_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_pre_regress.nc")
nIndR_ssp585_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_pre_regress.nc")
sIndR_ssp585_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_pre_regress.nc")
IndR_ssp585_p3_pre_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_pre_regress.nc")

### 环流场回归

In [ ]:
(
    nIndR_his_hgt_slope,
    nIndR_his_hgt_intercept,
    nIndR_his_hgt_rvalue,
    nIndR_his_hgt_pvalue,
    nIndR_his_hgt_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, hgthis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_his_u_slope,
    nIndR_his_u_intercept,
    nIndR_his_u_rvalue,
    nIndR_his_u_pvalue,
    nIndR_his_u_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, uhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_his_v_slope,
    nIndR_his_v_intercept,
    nIndR_his_v_rvalue,
    nIndR_his_v_pvalue,
    nIndR_his_v_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, vhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_his_hgt_slope,
    sIndR_his_hgt_intercept,
    sIndR_his_hgt_rvalue,
    sIndR_his_hgt_pvalue,
    sIndR_his_hgt_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, hgthis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_his_u_slope,
    sIndR_his_u_intercept,
    sIndR_his_u_rvalue,
    sIndR_his_u_pvalue,
    sIndR_his_u_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, uhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_his_v_slope,
    sIndR_his_v_intercept,
    sIndR_his_v_rvalue,
    sIndR_his_v_pvalue,
    sIndR_his_v_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, vhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_his_hgt_slope,
    IndR_his_hgt_intercept,
    IndR_his_hgt_rvalue,
    IndR_his_hgt_pvalue,
    IndR_his_hgt_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, hgthis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_his_u_slope,
    IndR_his_u_intercept,
    IndR_his_u_rvalue,
    IndR_his_u_pvalue,
    IndR_his_u_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, uhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_his_v_slope,
    IndR_his_v_intercept,
    IndR_his_v_rvalue,
    IndR_his_v_pvalue,
    IndR_his_v_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, vhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

In [ ]:
(
    nIndR_ssp585_p3_hgt_slope,
    nIndR_ssp585_p3_hgt_intercept,
    nIndR_ssp585_p3_hgt_rvalue,
    nIndR_ssp585_p3_hgt_pvalue,
    nIndR_ssp585_p3_hgt_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))
(
    sIndR_ssp585_p3_hgt_slope,
    sIndR_ssp585_p3_hgt_intercept,
    sIndR_ssp585_p3_hgt_rvalue,
    sIndR_ssp585_p3_hgt_pvalue,
    sIndR_ssp585_p3_hgt_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))
(
    IndR_ssp585_p3_hgt_slope,
    IndR_ssp585_p3_hgt_intercept,
    IndR_ssp585_p3_hgt_rvalue,
    IndR_ssp585_p3_hgt_pvalue,
    IndR_ssp585_p3_hgt_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

models = nIndR_ssp585_p3_hgt_slope.coords["models"]
level = nIndR_ssp585_p3_hgt_slope.coords["level"]

nIndR_ssp585_p3_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ssp585_p3 regress onto 1979-2014 "),
)

sIndR_ssp585_p3_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ssp585_p3 regress onto 1979-2014 "),
)

IndR_ssp585_p3_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ssp585_p3 regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc")
nIndR_ssp585_p3_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc")
sIndR_ssp585_p3_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc")
IndR_ssp585_p3_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc")

In [ ]:
(
    nIndR_ssp585_p3_hgt_slope,
    nIndR_ssp585_p3_hgt_intercept,
    nIndR_ssp585_p3_hgt_rvalue,
    nIndR_ssp585_p3_hgt_pvalue,
    nIndR_ssp585_p3_hgt_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_ssp585_p3_u_slope,
    nIndR_ssp585_p3_u_intercept,
    nIndR_ssp585_p3_u_rvalue,
    nIndR_ssp585_p3_u_pvalue,
    nIndR_ssp585_p3_u_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, ussp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    nIndR_ssp585_p3_v_slope,
    nIndR_ssp585_p3_v_intercept,
    nIndR_ssp585_p3_v_rvalue,
    nIndR_ssp585_p3_v_pvalue,
    nIndR_ssp585_p3_v_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, vssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ssp585_p3_hgt_slope,
    sIndR_ssp585_p3_hgt_intercept,
    sIndR_ssp585_p3_hgt_rvalue,
    sIndR_ssp585_p3_hgt_pvalue,
    sIndR_ssp585_p3_hgt_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ssp585_p3_u_slope,
    sIndR_ssp585_p3_u_intercept,
    sIndR_ssp585_p3_u_rvalue,
    sIndR_ssp585_p3_u_pvalue,
    sIndR_ssp585_p3_u_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, ussp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    sIndR_ssp585_p3_v_slope,
    sIndR_ssp585_p3_v_intercept,
    sIndR_ssp585_p3_v_rvalue,
    sIndR_ssp585_p3_v_pvalue,
    sIndR_ssp585_p3_v_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, vssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ssp585_p3_hgt_slope,
    IndR_ssp585_p3_hgt_intercept,
    IndR_ssp585_p3_hgt_rvalue,
    IndR_ssp585_p3_hgt_pvalue,
    IndR_ssp585_p3_hgt_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ssp585_p3_u_slope,
    IndR_ssp585_p3_u_intercept,
    IndR_ssp585_p3_u_rvalue,
    IndR_ssp585_p3_u_pvalue,
    IndR_ssp585_p3_u_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, ussp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

(
    IndR_ssp585_p3_v_slope,
    IndR_ssp585_p3_v_intercept,
    IndR_ssp585_p3_v_rvalue,
    IndR_ssp585_p3_v_pvalue,
    IndR_ssp585_p3_v_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, vssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]))

In [ ]:
level = nIndR_his_hgt_slope.coords["level"]
nIndR_his_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_his_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_his_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_his_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_his_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_his_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of his regress onto 1979-2014 "),
)

sIndR_his_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_his_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_his_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_his_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_his_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_his_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of his regress onto 1979-2014 "),
)

IndR_his_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_his_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_his_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_his_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_his_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_his_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of his regress onto 1979-2014 "),
)

nIndR_his_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_his_u_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_his_u_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_his_u_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_his_u_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_his_u_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of his regress onto 1979-2014 "),
)

sIndR_his_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_his_u_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_his_u_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_his_u_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_his_u_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_his_u_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of his regress onto 1979-2014 "),
)

IndR_his_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_his_u_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_his_u_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_his_u_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_his_u_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_his_u_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of his regress onto 1979-2014 "),
)

nIndR_his_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_his_v_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_his_v_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_his_v_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_his_v_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_his_v_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of his regress onto 1979-2014 "),
)

sIndR_his_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_his_v_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_his_v_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_his_v_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_his_v_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_his_v_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of his regress onto 1979-2014 "),
)

IndR_his_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_his_v_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_his_v_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_his_v_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_his_v_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_his_v_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of his regress onto 1979-2014 "),
)

In [ ]:
nIndR_ssp585_p3_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ssp585_p3 regress onto 1979-2014 "),
)

sIndR_ssp585_p3_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ssp585_p3 regress onto 1979-2014 "),
)

IndR_ssp585_p3_hgt_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_ssp585_p3_hgt_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt fields of ssp585_p3 regress onto 1979-2014 "),
)

nIndR_ssp585_p3_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_u_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_u_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_u_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_u_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_u_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of ssp585_p3 regress onto 1979-2014 "),
)

sIndR_ssp585_p3_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_u_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_u_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_u_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_u_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_u_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of ssp585_p3 regress onto 1979-2014 "),
)

IndR_ssp585_p3_u_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_ssp585_p3_u_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_ssp585_p3_u_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_u_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_u_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_ssp585_p3_u_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u fields of ssp585_p3 regress onto 1979-2014 "),
)

nIndR_ssp585_p3_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_v_slope.data),
        intercept=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_v_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_v_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_v_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], nIndR_ssp585_p3_v_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of ssp585_p3 regress onto 1979-2014 "),
)

sIndR_ssp585_p3_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_v_slope.data),
        intercept=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_v_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_v_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_v_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], sIndR_ssp585_p3_v_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of ssp585_p3 regress onto 1979-2014 "),
)

IndR_ssp585_p3_v_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "level", "lat", "lon"], IndR_ssp585_p3_v_slope.data),
        intercept=(["models", "level", "lat", "lon"], IndR_ssp585_p3_v_intercept.data),
        rvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_v_rvalue.data),
        pvalue=(["models", "level", "lat", "lon"], IndR_ssp585_p3_v_pvalue.data),
        hypothesis=(["models", "level", "lat", "lon"], IndR_ssp585_p3_v_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        level=level.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v fields of ssp585_p3 regress onto 1979-2014 "),
)

In [ ]:
# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_hgt_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc")

nIndR_his_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_hgt_regress.nc")
sIndR_his_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_hgt_regress.nc")
IndR_his_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_hgt_regress.nc")
nIndR_ssp585_p3_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc")
sIndR_ssp585_p3_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc")
IndR_ssp585_p3_hgt_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc")

In [ ]:
# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_u_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_u_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_u_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_u_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_u_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_u_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_u_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_u_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_u_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_u_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_u_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_u_regress.nc")

nIndR_his_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_u_regress.nc")
sIndR_his_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_u_regress.nc")
IndR_his_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_u_regress.nc")
nIndR_ssp585_p3_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_u_regress.nc")
sIndR_ssp585_p3_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_u_regress.nc")
IndR_ssp585_p3_u_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_u_regress.nc")

In [ ]:
# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_v_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_v_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_v_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_v_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_v_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_v_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_v_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_v_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_v_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_v_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_v_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_v_regress.nc")

nIndR_his_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_v_regress.nc")
sIndR_his_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_v_regress.nc")
IndR_his_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_v_regress.nc")
nIndR_ssp585_p3_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_v_regress.nc")
sIndR_ssp585_p3_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_v_regress.nc")
IndR_ssp585_p3_v_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_v_regress.nc")

### 垂直积分水汽通量

In [ ]:
(
    nIndR_his_uq_slope,
    nIndR_his_uq_intercept,
    nIndR_his_uq_rvalue,
    nIndR_his_uq_pvalue,
    nIndR_his_uq_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, uqhis_ver_JJA)

(
    nIndR_his_vq_slope,
    nIndR_his_vq_intercept,
    nIndR_his_vq_rvalue,
    nIndR_his_vq_pvalue,
    nIndR_his_vq_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, vqhis_ver_JJA)

(
    nIndR_his_uqvqdiv_slope,
    nIndR_his_uqvqdiv_intercept,
    nIndR_his_uqvqdiv_rvalue,
    nIndR_his_uqvqdiv_pvalue,
    nIndR_his_uqvqdiv_hypothesis,
) = ca.dim_linregress(prehis_nIndia_JJA_nor, uqvqdivhis_ver_JJA)

(
    sIndR_his_uq_slope,
    sIndR_his_uq_intercept,
    sIndR_his_uq_rvalue,
    sIndR_his_uq_pvalue,
    sIndR_his_uq_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, uqhis_ver_JJA)

(
    sIndR_his_vq_slope,
    sIndR_his_vq_intercept,
    sIndR_his_vq_rvalue,
    sIndR_his_vq_pvalue,
    sIndR_his_vq_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, vqhis_ver_JJA)

(
    sIndR_his_uqvqdiv_slope,
    sIndR_his_uqvqdiv_intercept,
    sIndR_his_uqvqdiv_rvalue,
    sIndR_his_uqvqdiv_pvalue,
    sIndR_his_uqvqdiv_hypothesis,
) = ca.dim_linregress(prehis_sIndia_JJA_nor, uqvqdivhis_ver_JJA)

(
    IndR_his_uq_slope,
    IndR_his_uq_intercept,
    IndR_his_uq_rvalue,
    IndR_his_uq_pvalue,
    IndR_his_uq_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, uqhis_ver_JJA)

(
    IndR_his_vq_slope,
    IndR_his_vq_intercept,
    IndR_his_vq_rvalue,
    IndR_his_vq_pvalue,
    IndR_his_vq_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, vqhis_ver_JJA)

(
    IndR_his_uqvqdiv_slope,
    IndR_his_uqvqdiv_intercept,
    IndR_his_uqvqdiv_rvalue,
    IndR_his_uqvqdiv_pvalue,
    IndR_his_uqvqdiv_hypothesis,
) = ca.dim_linregress(prehis_India_JJA_nor, uqvqdivhis_ver_JJA)

In [ ]:
(
    nIndR_ssp585_p3_uq_slope,
    nIndR_ssp585_p3_uq_intercept,
    nIndR_ssp585_p3_uq_rvalue,
    nIndR_ssp585_p3_uq_pvalue,
    nIndR_ssp585_p3_uq_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, uqssp585_p3_ver_JJA)

(
    nIndR_ssp585_p3_vq_slope,
    nIndR_ssp585_p3_vq_intercept,
    nIndR_ssp585_p3_vq_rvalue,
    nIndR_ssp585_p3_vq_pvalue,
    nIndR_ssp585_p3_vq_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, vqssp585_p3_ver_JJA)

(
    nIndR_ssp585_p3_uqvqdiv_slope,
    nIndR_ssp585_p3_uqvqdiv_intercept,
    nIndR_ssp585_p3_uqvqdiv_rvalue,
    nIndR_ssp585_p3_uqvqdiv_pvalue,
    nIndR_ssp585_p3_uqvqdiv_hypothesis,
) = ca.dim_linregress(pressp585_p3_nIndia_JJA_nor, uqvqdivssp585_p3_ver_JJA)

(
    sIndR_ssp585_p3_uq_slope,
    sIndR_ssp585_p3_uq_intercept,
    sIndR_ssp585_p3_uq_rvalue,
    sIndR_ssp585_p3_uq_pvalue,
    sIndR_ssp585_p3_uq_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, uqssp585_p3_ver_JJA)

(
    sIndR_ssp585_p3_vq_slope,
    sIndR_ssp585_p3_vq_intercept,
    sIndR_ssp585_p3_vq_rvalue,
    sIndR_ssp585_p3_vq_pvalue,
    sIndR_ssp585_p3_vq_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, vqssp585_p3_ver_JJA)

(
    sIndR_ssp585_p3_uqvqdiv_slope,
    sIndR_ssp585_p3_uqvqdiv_intercept,
    sIndR_ssp585_p3_uqvqdiv_rvalue,
    sIndR_ssp585_p3_uqvqdiv_pvalue,
    sIndR_ssp585_p3_uqvqdiv_hypothesis,
) = ca.dim_linregress(pressp585_p3_sIndia_JJA_nor, uqvqdivssp585_p3_ver_JJA)

(
    IndR_ssp585_p3_uq_slope,
    IndR_ssp585_p3_uq_intercept,
    IndR_ssp585_p3_uq_rvalue,
    IndR_ssp585_p3_uq_pvalue,
    IndR_ssp585_p3_uq_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, uqssp585_p3_ver_JJA)

(
    IndR_ssp585_p3_vq_slope,
    IndR_ssp585_p3_vq_intercept,
    IndR_ssp585_p3_vq_rvalue,
    IndR_ssp585_p3_vq_pvalue,
    IndR_ssp585_p3_vq_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, vqssp585_p3_ver_JJA)

(
    IndR_ssp585_p3_uqvqdiv_slope,
    IndR_ssp585_p3_uqvqdiv_intercept,
    IndR_ssp585_p3_uqvqdiv_rvalue,
    IndR_ssp585_p3_uqvqdiv_pvalue,
    IndR_ssp585_p3_uqvqdiv_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA_nor, uqvqdivssp585_p3_ver_JJA)

In [ ]:
nIndR_his_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_uq_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of his regress onto 1979-2014 "),
)
sIndR_his_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_uq_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of his regress onto 1979-2014 "),
)
IndR_his_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_uq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of his regress onto 1979-2014 "),
)

nIndR_ssp585_p3_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_uq_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ssp585_p3 regress onto 1979-2014 "),
)
sIndR_ssp585_p3_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_uq_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ssp585_p3 regress onto 1979-2014 "),
)
IndR_ssp585_p3_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_uq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ssp585_p3 regress onto 1979-2014 "),
)

# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uq_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uq_regress.nc")
  
nIndR_his_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uq_regress.nc")
sIndR_his_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uq_regress.nc")
IndR_his_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uq_regress.nc")
nIndR_ssp585_p3_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uq_regress.nc")
sIndR_ssp585_p3_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uq_regress.nc")
IndR_ssp585_p3_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uq_regress.nc")

In [ ]:
nIndR_his_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_vq_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of his regress onto 1979-2014 "),
)
sIndR_his_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_vq_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of his regress onto 1979-2014 "),
)
IndR_his_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_vq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of his regress onto 1979-2014 "),
)

nIndR_ssp585_p3_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_vq_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ssp585_p3 regress onto 1979-2014 "),
)
sIndR_ssp585_p3_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_vq_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ssp585_p3 regress onto 1979-2014 "),
)
IndR_ssp585_p3_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_vq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ssp585_p3 regress onto 1979-2014 "),
)

# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_vq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_vq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_vq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_vq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_vq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_vq_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_vq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_vq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_vq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_vq_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_vq_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_vq_regress.nc")
  
nIndR_his_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_vq_regress.nc")
sIndR_his_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_vq_regress.nc")
IndR_his_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_vq_regress.nc")
nIndR_ssp585_p3_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_vq_regress.nc")
sIndR_ssp585_p3_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_vq_regress.nc")
IndR_ssp585_p3_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_vq_regress.nc")

In [ ]:
nIndR_his_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of his regress onto 1979-2014 "),
)
sIndR_his_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of his regress onto 1979-2014 "),
)
IndR_his_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of his regress onto 1979-2014 "),
)

nIndR_ssp585_p3_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ssp585_p3 regress onto 1979-2014 "),
)
sIndR_ssp585_p3_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ssp585_p3 regress onto 1979-2014 "),
)
IndR_ssp585_p3_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ssp585_p3 regress onto 1979-2014 "),
)

# historical
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uqvqdiv_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uqvqdiv_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uqvqdiv_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uqvqdiv_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uqvqdiv_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uqvqdiv_regress.nc")

# ssp585_p3
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uqvqdiv_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uqvqdiv_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uqvqdiv_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uqvqdiv_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uqvqdiv_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uqvqdiv_regress.nc")
  
nIndR_his_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uqvqdiv_regress.nc")
sIndR_his_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uqvqdiv_regress.nc")
IndR_his_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uqvqdiv_regress.nc")
nIndR_ssp585_p3_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uqvqdiv_regress.nc")
sIndR_ssp585_p3_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uqvqdiv_regress.nc")
IndR_ssp585_p3_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uqvqdiv_regress.nc")

### 海温

In [ ]:
ssthis_MAM.name = "sst"
ssthis_JJA.name = "sst"
ssthis_SON.name = "sst"
ssthis_DJF.name = "sst"
sstssp585_p3_MAM.name = "sst"
sstssp585_p3_JJA.name = "sst"
sstssp585_p3_SON.name = "sst"
sstssp585_p3_DJF.name = "sst"

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = prehis_nIndia_JJA_nor[:,1:].copy()
    tmp.coords["time"] = ssthis_DJF.time
    (
        nIndR_his_sstDJF_slope,
        nIndR_his_sstDJF_intercept,
        nIndR_his_sstDJF_rvalue,
        nIndR_his_sstDJF_pvalue,
        nIndR_his_sstDJF_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_DJF.chunk(dict(time=-1)))
    (
        nIndR_his_sstDJF_slope,
        nIndR_his_sstDJF_intercept,
        nIndR_his_sstDJF_rvalue,
        nIndR_his_sstDJF_pvalue,
        nIndR_his_sstDJF_hypothesis,
    ) = dask.compute(
        nIndR_his_sstDJF_slope,
        nIndR_his_sstDJF_intercept,
        nIndR_his_sstDJF_rvalue,
        nIndR_his_sstDJF_pvalue,
        nIndR_his_sstDJF_hypothesis,
    )
    
    tmp = prehis_sIndia_JJA_nor[:,1:].copy()
    tmp.coords["time"] = ssthis_DJF.time
    (
        sIndR_his_sstDJF_slope,
        sIndR_his_sstDJF_intercept,
        sIndR_his_sstDJF_rvalue,
        sIndR_his_sstDJF_pvalue,
        sIndR_his_sstDJF_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_DJF.chunk(dict(time=-1)))
    (
        sIndR_his_sstDJF_slope,
        sIndR_his_sstDJF_intercept,
        sIndR_his_sstDJF_rvalue,
        sIndR_his_sstDJF_pvalue,
        sIndR_his_sstDJF_hypothesis,
    ) = dask.compute(
        sIndR_his_sstDJF_slope,
        sIndR_his_sstDJF_intercept,
        sIndR_his_sstDJF_rvalue,
        sIndR_his_sstDJF_pvalue,
        sIndR_his_sstDJF_hypothesis,
    )
    
    tmp = prehis_India_JJA_nor[:,1:].copy()
    tmp.coords["time"] = ssthis_DJF.time
    (
        IndR_his_sstDJF_slope,
        IndR_his_sstDJF_intercept,
        IndR_his_sstDJF_rvalue,
        IndR_his_sstDJF_pvalue,
        IndR_his_sstDJF_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_DJF.chunk(dict(time=-1)))
    (
        IndR_his_sstDJF_slope,
        IndR_his_sstDJF_intercept,
        IndR_his_sstDJF_rvalue,
        IndR_his_sstDJF_pvalue,
        IndR_his_sstDJF_hypothesis,
    ) = dask.compute(
        IndR_his_sstDJF_slope,
        IndR_his_sstDJF_intercept,
        IndR_his_sstDJF_rvalue,
        IndR_his_sstDJF_pvalue,
        IndR_his_sstDJF_hypothesis,
    )
client.close()

In [ ]:
models = hgthis_ver_JJA.coords["models"]
nIndR_his_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_sstDJF_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_sstDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_sstDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_sstDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_sstDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

sIndR_his_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_sstDJF_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_sstDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_sstDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_sstDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_sstDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

IndR_his_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_sstDJF_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_sstDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_sstDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_sstDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_sstDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstDJF_regress.nc")
nIndR_his_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstDJF_regress.nc")
sIndR_his_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstDJF_regress.nc")
IndR_his_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstDJF_regress.nc")

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = prehis_nIndia_JJA_nor.copy()
    tmp.coords["time"] = ssthis_MAM.time
    (
        nIndR_his_sstMAM_slope,
        nIndR_his_sstMAM_intercept,
        nIndR_his_sstMAM_rvalue,
        nIndR_his_sstMAM_pvalue,
        nIndR_his_sstMAM_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_MAM.chunk(dict(time=-1)))
    (
        nIndR_his_sstMAM_slope,
        nIndR_his_sstMAM_intercept,
        nIndR_his_sstMAM_rvalue,
        nIndR_his_sstMAM_pvalue,
        nIndR_his_sstMAM_hypothesis,
    ) = dask.compute(
        nIndR_his_sstMAM_slope,
        nIndR_his_sstMAM_intercept,
        nIndR_his_sstMAM_rvalue,
        nIndR_his_sstMAM_pvalue,
        nIndR_his_sstMAM_hypothesis,
    )
    
    tmp = prehis_sIndia_JJA_nor.copy()
    tmp.coords["time"] = ssthis_MAM.time
    (
        sIndR_his_sstMAM_slope,
        sIndR_his_sstMAM_intercept,
        sIndR_his_sstMAM_rvalue,
        sIndR_his_sstMAM_pvalue,
        sIndR_his_sstMAM_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_MAM.chunk(dict(time=-1)))
    (
        sIndR_his_sstMAM_slope,
        sIndR_his_sstMAM_intercept,
        sIndR_his_sstMAM_rvalue,
        sIndR_his_sstMAM_pvalue,
        sIndR_his_sstMAM_hypothesis,
    ) = dask.compute(
        sIndR_his_sstMAM_slope,
        sIndR_his_sstMAM_intercept,
        sIndR_his_sstMAM_rvalue,
        sIndR_his_sstMAM_pvalue,
        sIndR_his_sstMAM_hypothesis,
    )
    
    tmp = prehis_India_JJA_nor.copy()
    tmp.coords["time"] = ssthis_MAM.time
    (
        IndR_his_sstMAM_slope,
        IndR_his_sstMAM_intercept,
        IndR_his_sstMAM_rvalue,
        IndR_his_sstMAM_pvalue,
        IndR_his_sstMAM_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_MAM.chunk(dict(time=-1)))
    (
        IndR_his_sstMAM_slope,
        IndR_his_sstMAM_intercept,
        IndR_his_sstMAM_rvalue,
        IndR_his_sstMAM_pvalue,
        IndR_his_sstMAM_hypothesis,
    ) = dask.compute(
        IndR_his_sstMAM_slope,
        IndR_his_sstMAM_intercept,
        IndR_his_sstMAM_rvalue,
        IndR_his_sstMAM_pvalue,
        IndR_his_sstMAM_hypothesis,
    )
client.close()

In [ ]:
models = hgthis_ver_JJA.coords["models"]
nIndR_his_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_sstMAM_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_sstMAM_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_sstMAM_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_sstMAM_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_sstMAM_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

sIndR_his_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_sstMAM_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_sstMAM_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_sstMAM_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_sstMAM_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_sstMAM_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

IndR_his_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_sstMAM_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_sstMAM_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_sstMAM_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_sstMAM_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_sstMAM_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstMAM_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstMAM_regress.nc")
nIndR_his_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstMAM_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstMAM_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstMAM_regress.nc")
sIndR_his_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstMAM_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstMAM_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstMAM_regress.nc")
IndR_his_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstMAM_regress.nc")

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = prehis_nIndia_JJA_nor.copy()
    tmp.coords["time"] = ssthis_JJA.time
    (
        nIndR_his_sstJJA_slope,
        nIndR_his_sstJJA_intercept,
        nIndR_his_sstJJA_rvalue,
        nIndR_his_sstJJA_pvalue,
        nIndR_his_sstJJA_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_JJA.chunk(dict(time=-1)))
    (
        nIndR_his_sstJJA_slope,
        nIndR_his_sstJJA_intercept,
        nIndR_his_sstJJA_rvalue,
        nIndR_his_sstJJA_pvalue,
        nIndR_his_sstJJA_hypothesis,
    ) = dask.compute(
        nIndR_his_sstJJA_slope,
        nIndR_his_sstJJA_intercept,
        nIndR_his_sstJJA_rvalue,
        nIndR_his_sstJJA_pvalue,
        nIndR_his_sstJJA_hypothesis,
    )
    
    tmp = prehis_sIndia_JJA_nor.copy()
    tmp.coords["time"] = ssthis_JJA.time
    (
        sIndR_his_sstJJA_slope,
        sIndR_his_sstJJA_intercept,
        sIndR_his_sstJJA_rvalue,
        sIndR_his_sstJJA_pvalue,
        sIndR_his_sstJJA_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_JJA.chunk(dict(time=-1)))
    (
        sIndR_his_sstJJA_slope,
        sIndR_his_sstJJA_intercept,
        sIndR_his_sstJJA_rvalue,
        sIndR_his_sstJJA_pvalue,
        sIndR_his_sstJJA_hypothesis,
    ) = dask.compute(
        sIndR_his_sstJJA_slope,
        sIndR_his_sstJJA_intercept,
        sIndR_his_sstJJA_rvalue,
        sIndR_his_sstJJA_pvalue,
        sIndR_his_sstJJA_hypothesis,
    )
    
    tmp = prehis_India_JJA_nor.copy()
    tmp.coords["time"] = ssthis_JJA.time
    (
        IndR_his_sstJJA_slope,
        IndR_his_sstJJA_intercept,
        IndR_his_sstJJA_rvalue,
        IndR_his_sstJJA_pvalue,
        IndR_his_sstJJA_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_JJA.chunk(dict(time=-1)))
    (
        IndR_his_sstJJA_slope,
        IndR_his_sstJJA_intercept,
        IndR_his_sstJJA_rvalue,
        IndR_his_sstJJA_pvalue,
        IndR_his_sstJJA_hypothesis,
    ) = dask.compute(
        IndR_his_sstJJA_slope,
        IndR_his_sstJJA_intercept,
        IndR_his_sstJJA_rvalue,
        IndR_his_sstJJA_pvalue,
        IndR_his_sstJJA_hypothesis,
    )
client.close()

In [ ]:
models = hgthis_ver_JJA.coords["models"]
nIndR_his_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_sstJJA_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_sstJJA_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_sstJJA_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_sstJJA_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_sstJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

sIndR_his_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_sstJJA_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_sstJJA_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_sstJJA_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_sstJJA_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_sstJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

IndR_his_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_sstJJA_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_sstJJA_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_sstJJA_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_sstJJA_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_sstJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstJJA_regress.nc")
nIndR_his_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstJJA_regress.nc")
sIndR_his_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstJJA_regress.nc")
IndR_his_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstJJA_regress.nc")

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = prehis_nIndia_JJA_nor.copy()
    tmp.coords["time"] = ssthis_SON.time
    (
        nIndR_his_sstSON_slope,
        nIndR_his_sstSON_intercept,
        nIndR_his_sstSON_rvalue,
        nIndR_his_sstSON_pvalue,
        nIndR_his_sstSON_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_SON.chunk(dict(time=-1)))
    (
        nIndR_his_sstSON_slope,
        nIndR_his_sstSON_intercept,
        nIndR_his_sstSON_rvalue,
        nIndR_his_sstSON_pvalue,
        nIndR_his_sstSON_hypothesis,
    ) = dask.compute(
        nIndR_his_sstSON_slope,
        nIndR_his_sstSON_intercept,
        nIndR_his_sstSON_rvalue,
        nIndR_his_sstSON_pvalue,
        nIndR_his_sstSON_hypothesis,
    )
    
    tmp = prehis_sIndia_JJA_nor.copy()
    tmp.coords["time"] = ssthis_SON.time
    (
        sIndR_his_sstSON_slope,
        sIndR_his_sstSON_intercept,
        sIndR_his_sstSON_rvalue,
        sIndR_his_sstSON_pvalue,
        sIndR_his_sstSON_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_SON.chunk(dict(time=-1)))
    (
        sIndR_his_sstSON_slope,
        sIndR_his_sstSON_intercept,
        sIndR_his_sstSON_rvalue,
        sIndR_his_sstSON_pvalue,
        sIndR_his_sstSON_hypothesis,
    ) = dask.compute(
        sIndR_his_sstSON_slope,
        sIndR_his_sstSON_intercept,
        sIndR_his_sstSON_rvalue,
        sIndR_his_sstSON_pvalue,
        sIndR_his_sstSON_hypothesis,
    )
    
    tmp = prehis_India_JJA_nor.copy()
    tmp.coords["time"] = ssthis_SON.time
    (
        IndR_his_sstSON_slope,
        IndR_his_sstSON_intercept,
        IndR_his_sstSON_rvalue,
        IndR_his_sstSON_pvalue,
        IndR_his_sstSON_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_SON.chunk(dict(time=-1)))
    (
        IndR_his_sstSON_slope,
        IndR_his_sstSON_intercept,
        IndR_his_sstSON_rvalue,
        IndR_his_sstSON_pvalue,
        IndR_his_sstSON_hypothesis,
    ) = dask.compute(
        IndR_his_sstSON_slope,
        IndR_his_sstSON_intercept,
        IndR_his_sstSON_rvalue,
        IndR_his_sstSON_pvalue,
        IndR_his_sstSON_hypothesis,
    )
client.close()

In [ ]:
models = hgthis_ver_JJA.coords["models"]
nIndR_his_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_sstSON_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_sstSON_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_sstSON_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_sstSON_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_sstSON_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

sIndR_his_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_sstSON_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_sstSON_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_sstSON_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_sstSON_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_sstSON_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

IndR_his_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_sstSON_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_sstSON_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_sstSON_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_sstSON_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_sstSON_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstSON_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstSON_regress.nc")
nIndR_his_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstSON_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstSON_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstSON_regress.nc")
sIndR_his_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstSON_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstSON_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstSON_regress.nc")
IndR_his_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstSON_regress.nc")

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = prehis_nIndia_JJA_nor[:,:-1].copy()
    tmp.coords["time"] = ssthis_DJF.time
    (
        nIndR_his_sstnDJF_slope,
        nIndR_his_sstnDJF_intercept,
        nIndR_his_sstnDJF_rvalue,
        nIndR_his_sstnDJF_pvalue,
        nIndR_his_sstnDJF_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_DJF.chunk(dict(time=-1)))
    (
        nIndR_his_sstnDJF_slope,
        nIndR_his_sstnDJF_intercept,
        nIndR_his_sstnDJF_rvalue,
        nIndR_his_sstnDJF_pvalue,
        nIndR_his_sstnDJF_hypothesis,
    ) = dask.compute(
        nIndR_his_sstnDJF_slope,
        nIndR_his_sstnDJF_intercept,
        nIndR_his_sstnDJF_rvalue,
        nIndR_his_sstnDJF_pvalue,
        nIndR_his_sstnDJF_hypothesis,
    )
    
    tmp = prehis_sIndia_JJA_nor[:,:-1].copy()
    tmp.coords["time"] = ssthis_DJF.time
    (
        sIndR_his_sstnDJF_slope,
        sIndR_his_sstnDJF_intercept,
        sIndR_his_sstnDJF_rvalue,
        sIndR_his_sstnDJF_pvalue,
        sIndR_his_sstnDJF_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_DJF.chunk(dict(time=-1)))
    (
        sIndR_his_sstnDJF_slope,
        sIndR_his_sstnDJF_intercept,
        sIndR_his_sstnDJF_rvalue,
        sIndR_his_sstnDJF_pvalue,
        sIndR_his_sstnDJF_hypothesis,
    ) = dask.compute(
        sIndR_his_sstnDJF_slope,
        sIndR_his_sstnDJF_intercept,
        sIndR_his_sstnDJF_rvalue,
        sIndR_his_sstnDJF_pvalue,
        sIndR_his_sstnDJF_hypothesis,
    )
    
    tmp = prehis_India_JJA_nor[:,:-1].copy()
    tmp.coords["time"] = ssthis_DJF.time
    (
        IndR_his_sstnDJF_slope,
        IndR_his_sstnDJF_intercept,
        IndR_his_sstnDJF_rvalue,
        IndR_his_sstnDJF_pvalue,
        IndR_his_sstnDJF_hypothesis,
    ) = ca.dim_linregress(tmp, ssthis_DJF.chunk(dict(time=-1)))
    (
        IndR_his_sstnDJF_slope,
        IndR_his_sstnDJF_intercept,
        IndR_his_sstnDJF_rvalue,
        IndR_his_sstnDJF_pvalue,
        IndR_his_sstnDJF_hypothesis,
    ) = dask.compute(
        IndR_his_sstnDJF_slope,
        IndR_his_sstnDJF_intercept,
        IndR_his_sstnDJF_rvalue,
        IndR_his_sstnDJF_pvalue,
        IndR_his_sstnDJF_hypothesis,
    )
client.close()

In [ ]:
models = hgthis_ver_JJA.coords["models"]
nIndR_his_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_his_sstnDJF_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_his_sstnDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_his_sstnDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_his_sstnDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_his_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

sIndR_his_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_his_sstnDJF_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_his_sstnDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_his_sstnDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_his_sstnDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_his_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

IndR_his_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_sstnDJF_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_sstnDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_sstnDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_sstnDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of historical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstnDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstnDJF_regress.nc")
nIndR_his_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstnDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstnDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstnDJF_regress.nc")
sIndR_his_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstnDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstnDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstnDJF_regress.nc")
IndR_his_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstnDJF_regress.nc")

In [ ]:
prehis_nIndia_JJA_nor.time

In [ ]:
ssthis_DJF.time

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = pressp585_p3_nIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_DJF.time
    (
        nIndR_ssp585_p3_sstDJF_slope,
        nIndR_ssp585_p3_sstDJF_intercept,
        nIndR_ssp585_p3_sstDJF_rvalue,
        nIndR_ssp585_p3_sstDJF_pvalue,
        nIndR_ssp585_p3_sstDJF_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_DJF.chunk(dict(time=-1)))
    (
        nIndR_ssp585_p3_sstDJF_slope,
        nIndR_ssp585_p3_sstDJF_intercept,
        nIndR_ssp585_p3_sstDJF_rvalue,
        nIndR_ssp585_p3_sstDJF_pvalue,
        nIndR_ssp585_p3_sstDJF_hypothesis,
    ) = dask.compute(
        nIndR_ssp585_p3_sstDJF_slope,
        nIndR_ssp585_p3_sstDJF_intercept,
        nIndR_ssp585_p3_sstDJF_rvalue,
        nIndR_ssp585_p3_sstDJF_pvalue,
        nIndR_ssp585_p3_sstDJF_hypothesis,
    )
    
    tmp = pressp585_p3_sIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_DJF.time
    (
        sIndR_ssp585_p3_sstDJF_slope,
        sIndR_ssp585_p3_sstDJF_intercept,
        sIndR_ssp585_p3_sstDJF_rvalue,
        sIndR_ssp585_p3_sstDJF_pvalue,
        sIndR_ssp585_p3_sstDJF_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_DJF.chunk(dict(time=-1)))
    (
        sIndR_ssp585_p3_sstDJF_slope,
        sIndR_ssp585_p3_sstDJF_intercept,
        sIndR_ssp585_p3_sstDJF_rvalue,
        sIndR_ssp585_p3_sstDJF_pvalue,
        sIndR_ssp585_p3_sstDJF_hypothesis,
    ) = dask.compute(
        sIndR_ssp585_p3_sstDJF_slope,
        sIndR_ssp585_p3_sstDJF_intercept,
        sIndR_ssp585_p3_sstDJF_rvalue,
        sIndR_ssp585_p3_sstDJF_pvalue,
        sIndR_ssp585_p3_sstDJF_hypothesis,
    )
    
    tmp = pressp585_p3_India_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_DJF.time
    (
        IndR_ssp585_p3_sstDJF_slope,
        IndR_ssp585_p3_sstDJF_intercept,
        IndR_ssp585_p3_sstDJF_rvalue,
        IndR_ssp585_p3_sstDJF_pvalue,
        IndR_ssp585_p3_sstDJF_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_DJF.chunk(dict(time=-1)))
    (
        IndR_ssp585_p3_sstDJF_slope,
        IndR_ssp585_p3_sstDJF_intercept,
        IndR_ssp585_p3_sstDJF_rvalue,
        IndR_ssp585_p3_sstDJF_pvalue,
        IndR_ssp585_p3_sstDJF_hypothesis,
    ) = dask.compute(
        IndR_ssp585_p3_sstDJF_slope,
        IndR_ssp585_p3_sstDJF_intercept,
        IndR_ssp585_p3_sstDJF_rvalue,
        IndR_ssp585_p3_sstDJF_pvalue,
        IndR_ssp585_p3_sstDJF_hypothesis,
    )
client.close()

if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = pressp585_p3_nIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_MAM.time
    (
        nIndR_ssp585_p3_sstMAM_slope,
        nIndR_ssp585_p3_sstMAM_intercept,
        nIndR_ssp585_p3_sstMAM_rvalue,
        nIndR_ssp585_p3_sstMAM_pvalue,
        nIndR_ssp585_p3_sstMAM_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_MAM.chunk(dict(time=-1)))
    (
        nIndR_ssp585_p3_sstMAM_slope,
        nIndR_ssp585_p3_sstMAM_intercept,
        nIndR_ssp585_p3_sstMAM_rvalue,
        nIndR_ssp585_p3_sstMAM_pvalue,
        nIndR_ssp585_p3_sstMAM_hypothesis,
    ) = dask.compute(
        nIndR_ssp585_p3_sstMAM_slope,
        nIndR_ssp585_p3_sstMAM_intercept,
        nIndR_ssp585_p3_sstMAM_rvalue,
        nIndR_ssp585_p3_sstMAM_pvalue,
        nIndR_ssp585_p3_sstMAM_hypothesis,
    )
    
    tmp = pressp585_p3_sIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_MAM.time
    (
        sIndR_ssp585_p3_sstMAM_slope,
        sIndR_ssp585_p3_sstMAM_intercept,
        sIndR_ssp585_p3_sstMAM_rvalue,
        sIndR_ssp585_p3_sstMAM_pvalue,
        sIndR_ssp585_p3_sstMAM_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_MAM.chunk(dict(time=-1)))
    (
        sIndR_ssp585_p3_sstMAM_slope,
        sIndR_ssp585_p3_sstMAM_intercept,
        sIndR_ssp585_p3_sstMAM_rvalue,
        sIndR_ssp585_p3_sstMAM_pvalue,
        sIndR_ssp585_p3_sstMAM_hypothesis,
    ) = dask.compute(
        sIndR_ssp585_p3_sstMAM_slope,
        sIndR_ssp585_p3_sstMAM_intercept,
        sIndR_ssp585_p3_sstMAM_rvalue,
        sIndR_ssp585_p3_sstMAM_pvalue,
        sIndR_ssp585_p3_sstMAM_hypothesis,
    )
    
    tmp = pressp585_p3_India_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_MAM.time
    (
        IndR_ssp585_p3_sstMAM_slope,
        IndR_ssp585_p3_sstMAM_intercept,
        IndR_ssp585_p3_sstMAM_rvalue,
        IndR_ssp585_p3_sstMAM_pvalue,
        IndR_ssp585_p3_sstMAM_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_MAM.chunk(dict(time=-1)))
    (
        IndR_ssp585_p3_sstMAM_slope,
        IndR_ssp585_p3_sstMAM_intercept,
        IndR_ssp585_p3_sstMAM_rvalue,
        IndR_ssp585_p3_sstMAM_pvalue,
        IndR_ssp585_p3_sstMAM_hypothesis,
    ) = dask.compute(
        IndR_ssp585_p3_sstMAM_slope,
        IndR_ssp585_p3_sstMAM_intercept,
        IndR_ssp585_p3_sstMAM_rvalue,
        IndR_ssp585_p3_sstMAM_pvalue,
        IndR_ssp585_p3_sstMAM_hypothesis,
    )
client.close()

if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = pressp585_p3_nIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_JJA.time
    (
        nIndR_ssp585_p3_sstJJA_slope,
        nIndR_ssp585_p3_sstJJA_intercept,
        nIndR_ssp585_p3_sstJJA_rvalue,
        nIndR_ssp585_p3_sstJJA_pvalue,
        nIndR_ssp585_p3_sstJJA_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_JJA.chunk(dict(time=-1)))
    (
        nIndR_ssp585_p3_sstJJA_slope,
        nIndR_ssp585_p3_sstJJA_intercept,
        nIndR_ssp585_p3_sstJJA_rvalue,
        nIndR_ssp585_p3_sstJJA_pvalue,
        nIndR_ssp585_p3_sstJJA_hypothesis,
    ) = dask.compute(
        nIndR_ssp585_p3_sstJJA_slope,
        nIndR_ssp585_p3_sstJJA_intercept,
        nIndR_ssp585_p3_sstJJA_rvalue,
        nIndR_ssp585_p3_sstJJA_pvalue,
        nIndR_ssp585_p3_sstJJA_hypothesis,
    )
    
    tmp = pressp585_p3_sIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_JJA.time
    (
        sIndR_ssp585_p3_sstJJA_slope,
        sIndR_ssp585_p3_sstJJA_intercept,
        sIndR_ssp585_p3_sstJJA_rvalue,
        sIndR_ssp585_p3_sstJJA_pvalue,
        sIndR_ssp585_p3_sstJJA_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_JJA.chunk(dict(time=-1)))
    (
        sIndR_ssp585_p3_sstJJA_slope,
        sIndR_ssp585_p3_sstJJA_intercept,
        sIndR_ssp585_p3_sstJJA_rvalue,
        sIndR_ssp585_p3_sstJJA_pvalue,
        sIndR_ssp585_p3_sstJJA_hypothesis,
    ) = dask.compute(
        sIndR_ssp585_p3_sstJJA_slope,
        sIndR_ssp585_p3_sstJJA_intercept,
        sIndR_ssp585_p3_sstJJA_rvalue,
        sIndR_ssp585_p3_sstJJA_pvalue,
        sIndR_ssp585_p3_sstJJA_hypothesis,
    )
    
    tmp = pressp585_p3_India_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_JJA.time
    (
        IndR_ssp585_p3_sstJJA_slope,
        IndR_ssp585_p3_sstJJA_intercept,
        IndR_ssp585_p3_sstJJA_rvalue,
        IndR_ssp585_p3_sstJJA_pvalue,
        IndR_ssp585_p3_sstJJA_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_JJA.chunk(dict(time=-1)))
    (
        IndR_ssp585_p3_sstJJA_slope,
        IndR_ssp585_p3_sstJJA_intercept,
        IndR_ssp585_p3_sstJJA_rvalue,
        IndR_ssp585_p3_sstJJA_pvalue,
        IndR_ssp585_p3_sstJJA_hypothesis,
    ) = dask.compute(
        IndR_ssp585_p3_sstJJA_slope,
        IndR_ssp585_p3_sstJJA_intercept,
        IndR_ssp585_p3_sstJJA_rvalue,
        IndR_ssp585_p3_sstJJA_pvalue,
        IndR_ssp585_p3_sstJJA_hypothesis,
    )
client.close()

if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = pressp585_p3_nIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_SON.time
    (
        nIndR_ssp585_p3_sstSON_slope,
        nIndR_ssp585_p3_sstSON_intercept,
        nIndR_ssp585_p3_sstSON_rvalue,
        nIndR_ssp585_p3_sstSON_pvalue,
        nIndR_ssp585_p3_sstSON_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_SON.chunk(dict(time=-1)))
    (
        nIndR_ssp585_p3_sstSON_slope,
        nIndR_ssp585_p3_sstSON_intercept,
        nIndR_ssp585_p3_sstSON_rvalue,
        nIndR_ssp585_p3_sstSON_pvalue,
        nIndR_ssp585_p3_sstSON_hypothesis,
    ) = dask.compute(
        nIndR_ssp585_p3_sstSON_slope,
        nIndR_ssp585_p3_sstSON_intercept,
        nIndR_ssp585_p3_sstSON_rvalue,
        nIndR_ssp585_p3_sstSON_pvalue,
        nIndR_ssp585_p3_sstSON_hypothesis,
    )
    
    tmp = pressp585_p3_sIndia_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_SON.time
    (
        sIndR_ssp585_p3_sstSON_slope,
        sIndR_ssp585_p3_sstSON_intercept,
        sIndR_ssp585_p3_sstSON_rvalue,
        sIndR_ssp585_p3_sstSON_pvalue,
        sIndR_ssp585_p3_sstSON_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_SON.chunk(dict(time=-1)))
    (
        sIndR_ssp585_p3_sstSON_slope,
        sIndR_ssp585_p3_sstSON_intercept,
        sIndR_ssp585_p3_sstSON_rvalue,
        sIndR_ssp585_p3_sstSON_pvalue,
        sIndR_ssp585_p3_sstSON_hypothesis,
    ) = dask.compute(
        sIndR_ssp585_p3_sstSON_slope,
        sIndR_ssp585_p3_sstSON_intercept,
        sIndR_ssp585_p3_sstSON_rvalue,
        sIndR_ssp585_p3_sstSON_pvalue,
        sIndR_ssp585_p3_sstSON_hypothesis,
    )
    
    tmp = pressp585_p3_India_JJA_nor.copy()
    tmp.coords["time"] = sstssp585_p3_SON.time
    (
        IndR_ssp585_p3_sstSON_slope,
        IndR_ssp585_p3_sstSON_intercept,
        IndR_ssp585_p3_sstSON_rvalue,
        IndR_ssp585_p3_sstSON_pvalue,
        IndR_ssp585_p3_sstSON_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_SON.chunk(dict(time=-1)))
    (
        IndR_ssp585_p3_sstSON_slope,
        IndR_ssp585_p3_sstSON_intercept,
        IndR_ssp585_p3_sstSON_rvalue,
        IndR_ssp585_p3_sstSON_pvalue,
        IndR_ssp585_p3_sstSON_hypothesis,
    ) = dask.compute(
        IndR_ssp585_p3_sstSON_slope,
        IndR_ssp585_p3_sstSON_intercept,
        IndR_ssp585_p3_sstSON_rvalue,
        IndR_ssp585_p3_sstSON_pvalue,
        IndR_ssp585_p3_sstSON_hypothesis,
    )
client.close()

if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=6, threads_per_worker=5,memory_limit='100GB',processes=False)
    print(client.dashboard_link)
    # 前冬海温回归到降水
    tmp = pressp585_p3_nIndia_JJA_nor[:,:-1].copy()
    tmp.coords["time"] = sstssp585_p3_DJF[:,1:].time
    (
        nIndR_ssp585_p3_sstnDJF_slope,
        nIndR_ssp585_p3_sstnDJF_intercept,
        nIndR_ssp585_p3_sstnDJF_rvalue,
        nIndR_ssp585_p3_sstnDJF_pvalue,
        nIndR_ssp585_p3_sstnDJF_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_DJF[:,1:].chunk(dict(time=-1)))
    (
        nIndR_ssp585_p3_sstnDJF_slope,
        nIndR_ssp585_p3_sstnDJF_intercept,
        nIndR_ssp585_p3_sstnDJF_rvalue,
        nIndR_ssp585_p3_sstnDJF_pvalue,
        nIndR_ssp585_p3_sstnDJF_hypothesis,
    ) = dask.compute(
        nIndR_ssp585_p3_sstnDJF_slope,
        nIndR_ssp585_p3_sstnDJF_intercept,
        nIndR_ssp585_p3_sstnDJF_rvalue,
        nIndR_ssp585_p3_sstnDJF_pvalue,
        nIndR_ssp585_p3_sstnDJF_hypothesis,
    )
    
    tmp = pressp585_p3_sIndia_JJA_nor[:,:-1].copy()
    tmp.coords["time"] = sstssp585_p3_DJF[:,1:].time
    (
        sIndR_ssp585_p3_sstnDJF_slope,
        sIndR_ssp585_p3_sstnDJF_intercept,
        sIndR_ssp585_p3_sstnDJF_rvalue,
        sIndR_ssp585_p3_sstnDJF_pvalue,
        sIndR_ssp585_p3_sstnDJF_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_DJF[:,1:].chunk(dict(time=-1)))
    (
        sIndR_ssp585_p3_sstnDJF_slope,
        sIndR_ssp585_p3_sstnDJF_intercept,
        sIndR_ssp585_p3_sstnDJF_rvalue,
        sIndR_ssp585_p3_sstnDJF_pvalue,
        sIndR_ssp585_p3_sstnDJF_hypothesis,
    ) = dask.compute(
        sIndR_ssp585_p3_sstnDJF_slope,
        sIndR_ssp585_p3_sstnDJF_intercept,
        sIndR_ssp585_p3_sstnDJF_rvalue,
        sIndR_ssp585_p3_sstnDJF_pvalue,
        sIndR_ssp585_p3_sstnDJF_hypothesis,
    )
    
    tmp = pressp585_p3_India_JJA_nor[:,:-1].copy()
    tmp.coords["time"] = sstssp585_p3_DJF[:,1:].time
    (
        IndR_ssp585_p3_sstnDJF_slope,
        IndR_ssp585_p3_sstnDJF_intercept,
        IndR_ssp585_p3_sstnDJF_rvalue,
        IndR_ssp585_p3_sstnDJF_pvalue,
        IndR_ssp585_p3_sstnDJF_hypothesis,
    ) = ca.dim_linregress(tmp, sstssp585_p3_DJF[:,1:].chunk(dict(time=-1)))
    (
        IndR_ssp585_p3_sstnDJF_slope,
        IndR_ssp585_p3_sstnDJF_intercept,
        IndR_ssp585_p3_sstnDJF_rvalue,
        IndR_ssp585_p3_sstnDJF_pvalue,
        IndR_ssp585_p3_sstnDJF_hypothesis,
    ) = dask.compute(
        IndR_ssp585_p3_sstnDJF_slope,
        IndR_ssp585_p3_sstnDJF_intercept,
        IndR_ssp585_p3_sstnDJF_rvalue,
        IndR_ssp585_p3_sstnDJF_pvalue,
        IndR_ssp585_p3_sstnDJF_hypothesis,
    )
client.close()

In [ ]:
models = hgtssp585_p3_ver_JJA.coords["models"]
nIndR_ssp585_p3_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_sstDJF_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_sstDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_sstDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

sIndR_ssp585_p3_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_sstDJF_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_sstDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_sstDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

IndR_ssp585_p3_sstDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_sstDJF_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_sstDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_sstDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstDJF_regress.nc")
nIndR_ssp585_p3_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstDJF_regress.nc")
sIndR_ssp585_p3_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstDJF_regress.nc")
IndR_ssp585_p3_sstDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstDJF_regress.nc")

models = hgtssp585_p3_ver_JJA.coords["models"]
nIndR_ssp585_p3_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_sstMAM_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_sstMAM_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstMAM_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstMAM_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_sstMAM_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

sIndR_ssp585_p3_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_sstMAM_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_sstMAM_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstMAM_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstMAM_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_sstMAM_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

IndR_ssp585_p3_sstMAM_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_sstMAM_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_sstMAM_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstMAM_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstMAM_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_sstMAM_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstMAM_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstMAM_regress.nc")
nIndR_ssp585_p3_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstMAM_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstMAM_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstMAM_regress.nc")
sIndR_ssp585_p3_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstMAM_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstMAM_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstMAM_regress.nc")
IndR_ssp585_p3_sstMAM_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstMAM_regress.nc")
  
models = hgtssp585_p3_ver_JJA.coords["models"]
nIndR_ssp585_p3_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_sstJJA_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_sstJJA_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstJJA_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstJJA_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_sstJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

sIndR_ssp585_p3_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_sstJJA_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_sstJJA_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstJJA_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstJJA_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_sstJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

IndR_ssp585_p3_sstJJA_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_sstJJA_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_sstJJA_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstJJA_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstJJA_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_sstJJA_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstJJA_regress.nc")
nIndR_ssp585_p3_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstJJA_regress.nc")
sIndR_ssp585_p3_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstJJA_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstJJA_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstJJA_regress.nc")
IndR_ssp585_p3_sstJJA_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstJJA_regress.nc")


models = hgtssp585_p3_ver_JJA.coords["models"]
nIndR_ssp585_p3_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_sstSON_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_sstSON_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstSON_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstSON_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_sstSON_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

sIndR_ssp585_p3_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_sstSON_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_sstSON_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstSON_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstSON_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_sstSON_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

IndR_ssp585_p3_sstSON_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_sstSON_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_sstSON_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstSON_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstSON_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_sstSON_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstSON_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstSON_regress.nc")
nIndR_ssp585_p3_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstSON_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstSON_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstSON_regress.nc")
sIndR_ssp585_p3_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstSON_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstSON_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstSON_regress.nc")
IndR_ssp585_p3_sstSON_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstSON_regress.nc")

models = hgtssp585_p3_ver_JJA.coords["models"]
nIndR_ssp585_p3_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], nIndR_ssp585_p3_sstnDJF_slope.data),
        intercept=(["models", "lat", "lon"], nIndR_ssp585_p3_sstnDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstnDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], nIndR_ssp585_p3_sstnDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], nIndR_ssp585_p3_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

sIndR_ssp585_p3_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], sIndR_ssp585_p3_sstnDJF_slope.data),
        intercept=(["models", "lat", "lon"], sIndR_ssp585_p3_sstnDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstnDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], sIndR_ssp585_p3_sstnDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], sIndR_ssp585_p3_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

IndR_ssp585_p3_sstnDJF_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_sstnDJF_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_sstnDJF_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstnDJF_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_sstnDJF_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_sstnDJF_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sst fields of ssp585_p3torical regress onto 1979-2014 "),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstnDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstnDJF_regress.nc")
nIndR_ssp585_p3_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstnDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstnDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstnDJF_regress.nc")
sIndR_ssp585_p3_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstnDJF_regress.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstnDJF_regress.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstnDJF_regress.nc")
IndR_ssp585_p3_sstnDJF_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstnDJF_regress.nc")

### 南部-北部降水偏回归

In [ ]:
# historical
pre_his_north_south_slope, pre_his_north_south_pvalue = ca.multi_var_regression(prehis_JJA, [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
hgt_his_north_south_avalue, hgt_his_north_south_pvalue = ca.multi_var_regression(hgthis_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
u_his_north_south_avalue, u_his_north_south_pvalue = ca.multi_var_regression(uhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")
v_his_north_south_avalue, v_his_north_south_pvalue = ca.multi_var_regression(vhis_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor, prehis_sIndia_JJA_nor], concat_dim="area")

# ssp585
pre_ssp585_p3_north_south_slope, pre_ssp585_p3_north_south_pvalue = ca.multi_var_regression(pressp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
hgt_ssp585_p3_north_south_avalue, hgt_ssp585_p3_north_south_pvalue = ca.multi_var_regression(hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
u_ssp585_p3_north_south_avalue, u_ssp585_p3_north_south_pvalue = ca.multi_var_regression(ussp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")
v_ssp585_p3_north_south_avalue, v_ssp585_p3_north_south_pvalue = ca.multi_var_regression(vssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

In [45]:

# historical
tmp = ssthis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstDJF_his_north_south_slope, sstDJF_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = ssthis_MAM[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstMAM_his_north_south_slope, sstMAM_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = ssthis_JJA[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstJJA_his_north_south_slope, sstJJA_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = ssthis_SON[:,1:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,1:].time
sstSON_his_north_south_slope, sstSON_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:]], concat_dim="area")

tmp = ssthis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor[:,:-1].time
sstnDJF_his_north_south_slope, sstnDJF_his_north_south_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,:-1], prehis_sIndia_JJA_nor[:,:-1]], concat_dim="area")


# ssp585
tmp = sstssp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstDJF_ssp585_p3_north_south_slope, sstDJF_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = sstssp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstMAM_ssp585_p3_north_south_slope, sstMAM_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = sstssp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstJJA_ssp585_p3_north_south_slope, sstJJA_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = sstssp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstSON_ssp585_p3_north_south_slope, sstSON_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor], concat_dim="area")

tmp = sstssp585_p3_DJF[:,1:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor[:,1:].time
sstnDJF_ssp585_p3_north_south_slope, sstnDJF_ssp585_p3_north_south_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,1:], pressp585_p3_sIndia_JJA_nor[:,1:]], concat_dim="area")

In [ ]:
# historical
pre_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=pre_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=hgt_his_north_south_avalue.coords["models"].data,
        level=hgt_his_north_south_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_his_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=u_his_north_south_avalue.coords["models"].data,
        level=u_his_north_south_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_his_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=v_his_north_south_avalue.coords["models"].data,
        level=v_his_north_south_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)

# ssp585
pre_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=pre_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=hgt_ssp585_p3_north_south_avalue.coords["models"].data,
        level=hgt_ssp585_p3_north_south_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_ssp585_p3_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=u_ssp585_p3_north_south_avalue.coords["models"].data,
        level=u_ssp585_p3_north_south_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_ssp585_p3_north_south_avalue.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=v_ssp585_p3_north_south_avalue.coords["models"].data,
        level=v_ssp585_p3_north_south_avalue.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_his_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_his_north_south_partial_correlation.nc")

pre_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_partial_correlation.nc")
hgt_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_partial_correlation.nc")
u_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_his_north_south_partial_correlation.nc")
v_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_ssp585_p3_north_south_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_ssp585_p3_north_south_partial_correlation.nc")

pre_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_partial_correlation.nc")
hgt_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_partial_correlation.nc")
u_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_ssp585_p3_north_south_partial_correlation.nc")
v_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_ssp585_p3_north_south_partial_correlation.nc")

In [ ]:
# historical
sstDJF_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstDJF_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstMAM_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstJJA_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstSON_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_his_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstnDJF_his_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

# ssp585
sstDJF_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstDJF_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstMAM_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstJJA_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstSON_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_ssp585_p3_north_south_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south"]),
        models=sstnDJF_ssp585_p3_north_south_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_partial_correlation.nc")
sstDJF_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_partial_correlation.nc")
sstMAM_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_partial_correlation.nc")
sstJJA_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_partial_correlation.nc")
sstSON_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_partial_correlation.nc")
sstnDJF_his_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_partial_correlation.nc")

# ssp585
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_partial_correlation.nc")
sstDJF_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_partial_correlation.nc")
sstMAM_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_partial_correlation.nc")
sstJJA_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_partial_correlation.nc")
sstSON_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_partial_correlation.nc")
sstnDJF_ssp585_p3_north_south_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_partial_correlation.nc")

### 南部-北部-前冬Nino3.4 index偏回归

In [7]:
Nino34his_DJF_nor_copy = Nino34his_DJF_nor.copy()
Nino34his_DJF_nor_copy.coords["time"] = prehis_nIndia_JJA.time[1:]

pre_his_north_south_DJFnino34_slope, pre_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(prehis_JJA[:,1:,:,:], [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")

hgt_his_north_south_DJFnino34_slope, hgt_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(hgthis_ver_JJA[:,1:,:,:,:].sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")
u_his_north_south_DJFnino34_slope, u_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(uhis_ver_JJA[:,1:,:,:,:].sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")
v_his_north_south_DJFnino34_slope, v_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(vhis_ver_JJA[:,1:,:,:,:].sel(level=[200.0, 500.0, 850.0]), [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")

tmp = ssthis_DJF.copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor.time[1:]
sstDJF_his_north_south_DJFnino34_slope, sstDJF_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")

tmp = ssthis_MAM[:,1:,:,:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor.time[1:]
sstMAM_his_north_south_DJFnino34_slope, sstMAM_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")

tmp = ssthis_JJA[:,1:,:,:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor.time[1:]
sstJJA_his_north_south_DJFnino34_slope, sstJJA_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")

tmp = ssthis_SON[:,1:,:,:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor.time[1:]
sstSON_his_north_south_DJFnino34_slope, sstSON_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:], prehis_sIndia_JJA_nor[:,1:], Nino34his_DJF_nor_copy], concat_dim="area")

tmp = ssthis_DJF[:,1:,:,:].copy()
tmp.coords["time"] = prehis_nIndia_JJA_nor.time[1:-1]
sstnDJF_his_north_south_DJFnino34_slope, sstnDJF_his_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [prehis_nIndia_JJA_nor[:,1:-1], prehis_sIndia_JJA_nor[:,1:-1], Nino34his_DJF_nor_copy[:,:-1]], concat_dim="area")

In [8]:
Nino34ssp585_p3_DJF_nor_copy = Nino34ssp585_p3_DJF_nor.copy()
Nino34ssp585_p3_DJF_nor_copy.coords["time"] = pressp585_p3_nIndia_JJA.time

pre_ssp585_p3_north_south_DJFnino34_slope, pre_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(pressp585_p3_JJA, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")

hgt_ssp585_p3_north_south_DJFnino34_slope, hgt_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(hgtssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")
u_ssp585_p3_north_south_DJFnino34_slope, u_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(ussp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")
v_ssp585_p3_north_south_DJFnino34_slope, v_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(vssp585_p3_ver_JJA.sel(level=[200.0, 500.0, 850.0]), [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")


In [11]:
tmp = sstssp585_p3_DJF[:,1:,:,:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time[:-1]
sstnDJF_ssp585_p3_north_south_DJFnino34_slope, sstnDJF_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], pressp585_p3_sIndia_JJA_nor[:,:-1], Nino34ssp585_p3_DJF_nor_copy[:,:-1]], concat_dim="area")

In [ ]:
tmp = sstssp585_p3_DJF.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstDJF_ssp585_p3_north_south_DJFnino34_slope, sstDJF_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")

tmp = sstssp585_p3_MAM.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstMAM_ssp585_p3_north_south_DJFnino34_slope, sstMAM_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")

tmp = sstssp585_p3_JJA.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstJJA_ssp585_p3_north_south_DJFnino34_slope, sstJJA_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")

tmp = sstssp585_p3_SON.copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time
sstSON_ssp585_p3_north_south_DJFnino34_slope, sstSON_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor, pressp585_p3_sIndia_JJA_nor, Nino34ssp585_p3_DJF_nor_copy], concat_dim="area")

tmp = sstssp585_p3_DJF[:,1:,:,:].copy()
tmp.coords["time"] = pressp585_p3_nIndia_JJA_nor.time[:-1]
sstnDJF_ssp585_p3_north_south_DJFnino34_slope, sstnDJF_ssp585_p3_north_south_DJFnino34_pvalue = ca.multi_var_regression(tmp, [pressp585_p3_nIndia_JJA_nor[:,:-1], pressp585_p3_sIndia_JJA_nor[:,:-1], Nino34ssp585_p3_DJF_nor_copy[:,:-1]], concat_dim="area")

In [13]:
# historical
pre_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=pre_his_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=hgt_his_north_south_DJFnino34_slope.coords["models"].data,
        level=hgt_his_north_south_DJFnino34_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=u_his_north_south_DJFnino34_slope.coords["models"].data,
        level=u_his_north_south_DJFnino34_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=v_his_north_south_DJFnino34_slope.coords["models"].data,
        level=v_his_north_south_DJFnino34_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)

# ssp585
pre_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], pre_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=pre_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="precipitation partially regression onto north or south precipitation"),
)
hgt_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], hgt_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=hgt_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        level=hgt_ssp585_p3_north_south_DJFnino34_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="hgt partially regression onto north or south hgt"),
)

u_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], u_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], u_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=u_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        level=u_ssp585_p3_north_south_DJFnino34_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="u partially regression onto north or south u"),
)

v_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "level", "lat", "lon"], v_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "level", "lat", "lon"], v_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=v_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        level=v_ssp585_p3_north_south_DJFnino34_slope.coords["level"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="v partially regression onto north or south v"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_DJFnino34_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_DJFnino34_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_his_north_south_DJFnino34_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_his_north_south_DJFnino34_partial_correlation.nc")

pre_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_his_north_south_DJFnino34_partial_correlation.nc")
hgt_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_his_north_south_DJFnino34_partial_correlation.nc")
u_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_his_north_south_DJFnino34_partial_correlation.nc")
v_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_his_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")

pre_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/pre_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
hgt_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/hgt_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
u_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/u_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
v_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/v_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")

In [14]:
# historical
sstDJF_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstDJF_his_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstMAM_his_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstJJA_his_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstSON_his_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_his_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_his_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstnDJF_his_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

# ssp585
sstDJF_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstDJF_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstDJF_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstDJF partially regression onto north or south sstDJF"),
)

sstMAM_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstMAM_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstMAM_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstMAM partially regression onto north or south sstMAM"),
)

sstJJA_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstJJA_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstJJA_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstJJA partially regression onto north or south sstJJA"),
)

sstSON_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstSON_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstSON_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstSON partially regression onto north or south sstSON"),
)

sstnDJF_ssp585_p3_north_south_DJFnino34_partial_correlation = xr.Dataset(
    data_vars=dict(
        slope=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_DJFnino34_slope.transpose("area", ...).data),
        pvalue=(["area", "models", "lat", "lon"], sstnDJF_ssp585_p3_north_south_DJFnino34_pvalue.transpose("area", ...).data),
    ),
    coords=dict(
        area=np.array(["north", "south", "DJFNino3.4"]),
        models=sstnDJF_ssp585_p3_north_south_DJFnino34_slope.coords["models"].data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="sstnDJF partially regression onto north or south sstnDJF"),
)

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_DJFnino34_partial_correlation.nc")
sstDJF_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_his_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_DJFnino34_partial_correlation.nc")
sstMAM_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_his_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_DJFnino34_partial_correlation.nc")
sstJJA_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_his_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_DJFnino34_partial_correlation.nc")
sstSON_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_his_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_DJFnino34_partial_correlation.nc")
sstnDJF_his_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_his_north_south_DJFnino34_partial_correlation.nc")

# ssp585
if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
sstDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
sstMAM_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstMAM_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
sstJJA_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstJJA_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
sstSON_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstSON_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")

if os.path.exists("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.nc"):
  os.remove("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")
sstnDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/sstnDJF_ssp585_p3_north_south_DJFnino34_partial_correlation.nc")